# Stitching

This vignette will pilot stitching using BigStitcher, and demonstrate how to use these tools to stich a set of fields of view of in-situ sequencing data. 

Deep wants a picture that describes when things matter. 

* shifts within fields of view
* shifts across fields of view
* shifts across rounds

Intuition: you're dealing with RNA-scope folks // smFISH folks, and these represent inferior approaches. 
* stitching problem: reconciling adjacent fields of view with overlapping data
* stitching is a pre-requisite to solve registration problems whenever you have substantial shifts within a field of view.     

API proposal: 
* for each field of view, have an affine registration 
* dapi as anchor -> stitch together
* register across rounds 
* # fovs * # rounds matrices
* outputs a text file
* transform object has a transform.load method
* define a contract for loading such transforms
* create a proof of concept for applying transformations to point clouds
* transform images instead of point clouds? 

In [1]:
import starfish.data
experiment = starfish.data.ISS()

In [ ]:
Stitch dots image, then register each tile to the stitched dots image. 

In [3]:
experiment

<starfish.Experiment (FOVs=16)>
{
fov_000: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, c: 4, r: 4, x: 1390, y: 1044)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
    dots: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
fov_001: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, c: 4, r: 4, x: 1390, y: 1044)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
    dots: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
fov_002: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, c: 4, r: 4, x: 1390, y: 1044)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
    dots: <slicedimage.TileSet (z: 1, c: 1, r: 1, x: 1390, y: 1044)>
fov_003: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, c: 4, r: 4, x: 1390, y: 1044)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (

In [13]:
nuclei_images = {fov_name: fov.get_image("nuclei") for fov_name, fov in experiment.items()}

In [17]:
# to use this in BigStitcher, we want to save these data to disk.
from skimage.io import imsave
from os import makedirs
makedirs("images", exist_ok=True)
for fov, image in nuclei_images.items():
    imsave(f"images/dapi_{fov}.tiff", image.xarray.values)

100%|██████████| 1/1 [00:00<00:00, 42.79it/s]
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: images/dapi_fov_013.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
100%|██████████| 1/1 [00:00<00:00, 42.51it/s]


BigStitcher accepts TIFF files. Huzzah. It's got a manual loader. 

See https://imagej.net/BigStitcher_StackLoader

Choose Manual Loader. 

Select "No" for all options on the first page EXCEPT "tile", select Yes. -- we have one image per file. 

The image pattern should be `dapi_fov_{x}.tiff` where `x` is the tile number. 
The tiles should be `000,001,002,003,...,015` (elipses filled by sequential numbers between `3` and `15`)
This will build a BigDataViewer object. However, it doesn't have the tile positions. So, how do we get that information? Probably we just write this stupid XML file manually.

An alternative: It will accept a TileConfiguration file with the following format: 
https://imagej.net/Image_Stitching#Problems.2C_known_issues_and_solutions

```
# Define the number of dimensions we are working on
dim = 3
# Define the image coordinates (in pixels)
img_73.tif; ; (0.0, 0.0, 0.0)
img_74.tif; ; (409.0, 0.0, 0.0)
img_75.tif; ; (0.0, 409.0, 0.0)
img_76.tif; ; (409.0, 409.0, 0.0)
img_77.tif; ; (0.0, 818.0, 0.0)
img_78.tif; ; (409.0, 818.0, 0.0)
```

BigStitcher accepts a Tile Position txt file to organize the initial locations of the tiles.
It also accepts tile organization patterns, here we started the dataset as a rectangular grid first going right, then down. (initial position in the top left)

Next we right click and select "resave as hdf5" which promises better performance. 

Finally we can start stitching. 

We use bigstitcher to do affine translational registration only

In [106]:
!cat images/dataset.xml

In [29]:
import sys
!{sys.executable} -m pip install lxml

  Using cached https://files.pythonhosted.org/packages/bd/9f/6cda4672d3ad1aa4cf818ab8401a763787efba751c88aaf4b38fc8f923bb/lxml-4.4.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [85]:
import lxml
import numpy as np

In [76]:
tree = lxml.etree.parse("images/dataset.xml")
root = tree.getroot()
registrations = root[2].getchildren()

In [96]:
?np.fromstring

In [90]:
?np.genfromtxt

In [100]:
transforms = {}
for view in registrations:
    fov = view.get("setup")
    stitching_transform = view[0]
    name, transform = stitching_transform
    assert name.text == "Stitching Transform"
    affine_transform = np.fromstring(transform.text, dtype=float, sep=" ")
    transforms[int(fov)] = affine_transform.reshape(3, 4)    

In [102]:
transforms[4]

array([[  1.        ,   0.        ,   0.        ,  22.50790076],
       [  0.        ,   1.        ,   0.        , -49.8182063 ],
       [  0.        ,   0.        ,   1.        ,   0.        ]])

These are 3d affine transform matrices. See: https://people.cs.clemson.edu/~dhouse/courses/401/notes/affines-matrices.pdf

## Can we also register across rounds?

* save out max projections of each round
* register max projections against an anchor round (say, round 0)

In [ ]:
nuclei_images = {fov_name: fov.get_image("nuclei") for fov_name, fov in experiment.items()}

In [111]:
import starfish
from starfish.types import Axes
import os

In [117]:
os.makedirs("images_rounds", exist_ok=True)
mp = starfish.image.Filter.MaxProject({Axes.ROUND})
for fov_name, fov in experiment.items():
    img = fov.get_image("primary")
    projected = mp.run(img)
    data = projected.xarray.values
    for r in range(4):  # there are better ways to get the # rounds
        imsave(f"images_rounds/fov_{fov_name}_r_{r}.tiff", data[:, r].squeeze())

100%|██████████| 4/4 [00:00<00:00, 200.43it/s]
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: images_rounds/fov_fov_004_r_2.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
100%|██████████| 4/4 [00:00<00:00, 198.16it/s]
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: images_rounds/fov_fov_009_r_2.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
100%|██████████| 4/4 [00:00<00:00, 207.93it/s]
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: images_rounds/fov_fov_010_r_2.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)
100%|██████████| 4/4 [00:00<00:00, 204.27it/s]
/Users/ajc/projects/spacetx/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: images_rounds/fov_fov_011_r_2.tiff is a low contrast image
  warn('%s is

To stitch multiple rounds together, we need to use multi-view registration.
* We begin by stitching one round together. This sometimes goes badly. Does fourier cross-correlation change based on intensity? Do we need to run an intensity correction first?

Our data just have too many weird problems -- max projections are hard things to learn over. We should learn stitching from DAPI in each round or from a dots image. MERFISH might have this, let's try that dataset instead. 

Workflow appears to be: 
* https://imagej.net/BigStitcher#Documentation
* https://imagej.net/BigStitcher_Stitching_Mode
* https://imagej.net/BigStitcher_Pairwise_shift
* https://imagej.net/BigStitcher_Preview_Pairwise_shift
* https://imagej.net/BigStitcher_Global_optimization  # this one is important -- we need pairwise shifts across rounds ("compare") and I haven't figured that out yet. Might need to group the Tiles within time points?